# BeeHero Data Analyst Challenge

Resources for Linear Programming:

- Geometrical explanation: https://www.youtube.com/watch?v=E72DWgKP_1Y&t=841s
- scipy optimise docs Linear prog and integer linear prog: https://docs.scipy.org/doc/scipy/tutorial/optimize.html#linear-programming-example

## Background
BeeHero helps farmers (growers) get the bees they need for crop pollination. Here's what you need to know:

**What we do**: We connect beekeepers with farmers who need bees to pollinate their crops.

**How it works**: Farmers order a specific number of "bee frames" per acre of crop.

**What is a bee frame?**: A wooden structure where bees build honeycombs; each beehive contains up to 20 frames.

**Our goal**: Ensure farmers have exactly the right number of bee frames by commitment date which is February 15, 2024 (for the purpose of this exercise).

**The challenge**: Too few frames = unhappy farmers; too many frames = wasted resources and money.

## Database Overview
You'll be working with a SQLite database that contains these tables: <span style="color: red;">maybe use DB Browser for SQLite  (DB4S) to view db</span>

**sensors**: Information about electronic devices attached to beehives that collect data such as temperature, humidity and audio.

Each sensor monitors one beehive  <span style="color: red;"> can there be multiple sensors for one beehive?</span>  
Links to specific beekeepers (via group_id).  
<span style="color: red;">relational dbs/ table joins using primary key group_id</span>

**orchards**: The individual fields or areas within *larger* farms. Contains the connection between orchard and farm as well as our commitment for frames.  


**sensor_daily_statuses**: Shows which sensors are active in which orchards each day. If a sensor has no orchard ID, it's not being used this season.  
<span style="color: red;">Assuming boolean type.  
    Are there different sensors for each season? If yes, they stay installed the whole year? assuming there is no chance of damage over the course of the year if they remain in the beehive.</span>

**hive_updates**: Contains predictions about how many frames in each hive are covered by bees.  
<span style="color: red;">
    Note: two separate frames of bees visit fewer flowers on average compared to two frames put together, leading to stronger "pollination power" when they are combined.  
    <br>
    Possible reasons:
    <ul>
        <li style="color: red;">Two independent frames lead to each bee colony delegating tasks within its limited space, compared to when they are joined. This results in fewer bees allocated for 'foraging'.</li>
        <li style="color: red;">Each frame has a queen so more resources are needed to maintain.</li>
        <li style="color: red;">Larger cluster of bees leads to improved communication and improved foraging</li>
    </ul>
</span>


Today's date is February 5, 2024 (for the purpose of this exercise).  
<span style="color: red;">ten days before commitment on the 15th</span>

**Beekeeper**: Identified by group_id in the database.

**Growth rate**: Bee populations grow by approximately 0.2 frames per day (up to 20 frames maximum per hive).  
    <ul>
        <li style="color: red;"> Linear coefficient for the rate   </li>
        <li style="color: red;"> Assuming that based off the time range (10 days) and the rate of 0.2 per day, we can optimise the starting quantity of bees.</li>
        <li style="color: red;"> Why dont we start with 20 frames worth of bees from the beginning? Answer: leads colony to swarm and the queen leaves with half the bees to another location, and another queen is created. Will lead to reduced pollination. Unnatural to start with 20.</li>
        <li style="color: red;"> Expensive to buy a ready made 20 frame worth of bees </li>
        <li style="color: red;"> Easier to maintain optimal hive temperature with less initial bees </li>
    </ul>


**Valid hive update**: Data from within the last 5 days; older data is considered unreliable.

**Valid sensor status**: Similar to hive updates, a sensor's daily status is only considered valid if it is from within the last 5 days; older status data should not be used in your analysis.
<li style="color: red;"> FILTER DATA FOR WITHIN 5 DAYS </li>


## Access & Instructions

* Use the SQLite database provided [here](https://drive.google.com/file/d/1ZDX0LdstEZe-Brd3cHYnMqcdTT0FqmRk/view?usp=drive_link)
* You have **3 hours** to complete this challenge.
* For questions 1-5, focus on writing clear SQL queries or Python code. <span style="color: red;"> maybe do both </span>
* For question 6, provide Python code (you may use any library you want).
* Please define reusable code/queries during the different questions. <span style="color: red;"> OOP </span>
* Please document your approach.
* Don't worry about domain expertise - focus on demonstrating your data analysis skills

In [1]:
# Setup code - Run this first!
import sqlite3
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# Connect to database
# Replace with the path to your downloaded database file
conn = sqlite3.connect(r"C:\Users\samfi\OneDrive\Documents\Interviews_2025\home_assignment_data.db")


# Helper function for running queries
def run_query(query):
    return pd.read_sql(query, conn)


# Explore database structure
tables = run_query("SELECT name FROM sqlite_master WHERE type='table';")
print("Available tables:")
print(tables)

Available tables:
                    name
0                sensors
1               orchards
2  sensor_daily_statuses
3           hive_updates
4      temp_days_to_comm


## Table Notes:

- General Notes:
    - Current Date: February 5, 2024
    - Commitment Date: Feb 15th 2024
    - 1 beehive has max 20 frames  
    
<br>

- Sensors:
    - id: assuming is unique sensor (check if active)
    - Created: time of row creation (maybe sensor creation)
    - group_id: group of beekeepers (more than one beekeeper in group? assuming is 1 beekeeper. How many beekeepers per orchard/farm?)
    - is_locked_to_group: boolean, 0: Not locked so sensor can move groups, 1: sensor cannot move group of beekeepers
    - **One sensor per beehive** (Assumption that its 1-1) so total active sensors is total beehives?
- Orchards:
    - id: unique identifier for orchard (checked with SQL)
    - farm_id: farm the orchard is connected to
    - created: date of orchard creation
    - acres_pollinated: is this total acres in orchard or total pollinated or covered by bees?
    - commited_frames_per_acre: need to find how many acres in orchard/farm
    - Assuming REMOVE if commited is NULL
    
**For statuses, only take those from within 5 days, and only take most recent**
- Sensor_daily_statuses:
    - id/created/date: identifiers for status
    - sensor_id: join with id in sensors table
    - yard_id: WHAT IS YARD?
    - Orchard_id: join with id in orchards table. No id means sensor not in use this season
    - last_sampled: time when checked (always earlier than status creation, checked in SQL)
- hive_updates:
    - id/ created/date: identifier for status
    - bee_frames: number of bee frames in hive - assuming is how many are covered by bees rather than total. Grows by 0.2 per day average
    - sensor_id: sensor id for the hive. join with in sensors or sensor_id in sensor daily statuses.

## Part 1: Data Analysis Questions (90 minutes)

### Question 1: Active Sensors

How many sensors are active?

**Hint**: Use latest valid sensor daily status

In [2]:
# Your SQL query here
# Starter template:
query_1 = """
-- Find latest status of each sensor
-- Keep only valid statuses
-- Count sensors with valid orchard_id
SELECT COUNT(DISTINCT(sensor_id)) as active_sensors
FROM sensor_daily_statuses
WHERE created >= '2024-02-01 00:00:00'
AND orchard_id is not NULL
"""
#Notes:
# take latest from within 5 days (current date feb 5th 2024) for each sensor_id
# if older than 5 days than invalid status
# orchard_id not null
# do we take from end of feb 5th or from latest time on 5th (18:03). Assuming from the end of 5th as 
# status creation may not be current time. So date is 1,2,3,4,5
# could set date as variable rather than hard coded

# if latest_sample is NULL but the status has an orchard_id, is the status valid?? ASSUMPTION is yes
# if use where clause on latest_sample, it removes the NULL statuses, DO NOT WANT THIS
# Note: do i take created column or the last_sampled? There are cases where latest_sample is NULL and it has 
#orchard_id, so we want to include these sensors as valid. So we take the created.

# do not need groupby statement or the latest data as taking distinct sensor ids where their data is valid/active

result = run_query(query_1)
print(result)

   active_sensors
0           40251


### Question 2: Active Beekeepers

How many different beekeepers currently have hives deployed for pollination?

**Hint**: Count distinct group_ids from sensors that are actively participating

In [3]:
# Your SQL query here
query_2 = """
-- Count active beekeepers
-- Remember: Active means sensors are deployed in orchards with valid status
SELECT 
COUNT(DISTINCT group_id) as total_beekeepers_with_hives_for_pollination
FROM sensor_daily_statuses sds
JOIN sensors s
	ON sds.sensor_id = s.id
WHERE sds.created >= '2024-02-01 00:00:00'
AND sds.orchard_id is not NULL

"""

#Notes:
# Assumption: each unique group_id is a unique beekeeper
# Each sensor assigned one group_id/beekeeper
# The query first filters for active ids
# Then joins id in sensors with sensor_id in sensor_statuses
# We want to ensure each active sensor has a valid group_id (assuming they will as they are active) but just to be sure,
# we will do an inner Join rather than LEFT (to avoid null group_ids if there are)
# Assuming one beekeeper is assigned to each sensor because ids in sensors table are unique
# MANY sensor_ids to ONE group_id relationship 

# Do not need to include distinct sensor_id in the query as any valid sensor id is joined with the sensors table and we want
# the distinct group_ids that come with this.


result = run_query(query_2)
print(result)

   total_beekeepers_with_hives_for_pollination
0                                           56


### Question 3: Missing Data

How many active sensors don't have recent (valid) hive updates?

**Hint**: Find sensors that are active but don't have updates within the last 5 days

In [4]:
# Your SQL query here
query_3 = """
SELECT COUNT(DISTINCT(sensor_id)) as active_sensors_without_valid_update
FROM (

SELECT 
sensor_id,
MAX(created) as latest_status
FROM sensor_daily_statuses
WHERE orchard_id is not NULL
GROUP BY sensor_id)

WHERE latest_status < '2024-02-01 00:00:00'
"""
# same as 1 but filter for when last_sample for all statuses for the sensor is older than 5 days
# find latest last_sample and count those that are older than 5 days
# last_sampled Null removed with where latest_sample clause
# CHECK: total active sensors is 40940. 689 + 40251 = 40940 

result = run_query(query_3)
print(result)

   active_sensors_without_valid_update
0                                  689


### Question 4: Current Deployment Status

How many farms have:
- Too many bee frames (over-deployed)
- Too few bee frames (under-deployed)

**Expected output format**:
```
deployment_status | farm_count
over-deployed    | X
under-deployed   | Y
exact            | Z
```

In [5]:
# Your SQL query here
query_4 = """
-- Calculate deployment status for each farm
-- Compare frames_commitment in orchards table
-- Sum up actual frames from valid hive_updates
-- Group by farm and categorize status


WITH ranked_hive_updates as 
(
SELECT *,
ROW_NUMBER() OVER (PARTITION BY sensor_id ORDER BY created DESC) AS ranks
-- PARTITION BY sensor_id: creates groups by sensor_id
-- ROW_NUMBER(): assigns 1 to the latest hive update (desc means newest first)
-- OVER: defines the window to perform row_number over each group
FROM hive_updates
WHERE created >= '2024-02-01 00:00:00'
), farm_totals as 
( 
SELECT DISTINCT 
o.farm_id,
SUM(bee_frames) as total_bee_frames_in_farm,
MAX(o.acres_pollinated * o.committed_frames_per_acre) as total_commitment
FROM ranked_hive_updates
JOIN sensor_daily_statuses as sds
	USING(sensor_id)
JOIN orchards as o
	ON sds.orchard_id = o.id
WHERE ranks = 1
AND sds.orchard_id is not NULL 
GROUP BY o.farm_id
)
SELECT deployment_status, COUNT(*) as farm_count
FROM (
SELECT 
CASE 
	WHEN total_bee_frames_in_farm > total_commitment THEN 'over-deployed'
	WHEN total_bee_frames_in_farm < total_commitment THEN 'under-deployed'
	WHEN total_bee_frames_in_farm = total_commitment THEN 'exact'
	ELSE NULL 
END as deployment_status
FROM farm_totals
) as Deployment
GROUP BY deployment_status;
"""
#NOTES:
# Frames committed is per acre, so need to multiply by total acres in farm to find total commitment
# ASSUMPTION: acres_pollinated is how many acres in the orchard that need to be covered by the committed frames to be pollinated
# So, total commitmment is committed_frames_per_acre * acres_pollinated

# Plan:
# 1.Find latest and valid hive updates and use as a CTE with the WITH statement
# 2.Join that with sensor_daily_statuses (on sensor_id) to find orchard_id (another CTE)
# 3.Join with orchards via orchard id and then groupby farm_id
# 4.Use Case to assign farms to deployment status and then groupby deployment status and count occurrences of each status

# 1.
# Need to ensure the bee_frames value is corresponding to the latest created status in hive_updates (use window function)

# 2.
# When joining to sensor_status table, I am assuming orchard_id remains the same for each sensor_id, so 
# do not have to find latest sensor_status
# IMPORTANT NOTE: There are many sensors with no corresponding orchard_id in sensor_statuses so cannot assign to orchard 
# and thus farm, eg sensor 15562 has hive update but no sensor status. Assuming this sensor is invalid/ different season

# 3.
# Find total commitment via: orchards.acres_pollinated * orchards.committed_frames_per_acre
# find total_bee_frames_per_farm by performing groupby on farm_id and summing the bee_frames from each sensor in the farm
# Note: one sensor is one hive


# many less farms by the end compared to total farms, assuming we dont use a lot of farms in this season
result = run_query(query_4)
print(result)

  deployment_status  farm_count
0     over-deployed         149
1    under-deployed          35


### Question 5: Future Deployment Status

If we account for bee population growth (0.2 frames/day until Feb 15), how does the answer to question 4 change?

**Note**: Try not to use hardcoded values for the date.

In [6]:
# Your SQL query here
import pandas as pd
import datetime

#Needed to use python to find data differences because there doesnt seem to be a compatible operation to subtract 
#two dates in SQLite (maybe julianday) so lets use python. 

rank_query = """
SELECT *,
ROW_NUMBER() OVER (PARTITION BY sensor_id ORDER BY created DESC) AS ranks
-- PARTITION BY sensor_id: creates groups by sensor_id
-- ROW_NUMBER(): assigns 1 to the latest hive update (desc means newest first)
-- OVER: defines the window to perform row_number over each group
FROM hive_updates
WHERE created >= '2024-02-01 00:00:00'
"""
commitment_date = pd.to_datetime('2024-02-15')
df = pd.read_sql_query(rank_query, conn)
df['created'] = pd.to_datetime(df['created'])
df['bee_frames'] = df['bee_frames'] + 0.2*((commitment_date - df['created']).dt.days) # increased bee_frames from the growth rate
df['bee_frames'][df['bee_frames'] > 20] = 20 # ensure no hive has more than 20 bee frames
df.to_sql('temp_days_to_comm', conn, if_exists='replace', index=False)

query_5 = """
WITH ranked_hive_updates as 
(
SELECT * from temp_days_to_comm
), farm_totals as 
( 
SELECT DISTINCT 
o.farm_id,
SUM(bee_frames) as total_bee_frames_in_farm,
MAX(o.acres_pollinated * o.committed_frames_per_acre) as total_commitment
FROM ranked_hive_updates
JOIN sensor_daily_statuses as sds
	USING(sensor_id)
JOIN orchards as o
	ON sds.orchard_id = o.id
WHERE ranks = 1
AND sds.orchard_id is not NULL 
GROUP BY o.farm_id
)
SELECT deployment_status, COUNT(*) as farm_count
FROM (
SELECT 
CASE 
	WHEN total_bee_frames_in_farm > total_commitment THEN 'over-deployed'
	WHEN total_bee_frames_in_farm < total_commitment THEN 'under-deployed'
	WHEN total_bee_frames_in_farm = total_commitment THEN 'exact'
	ELSE NULL 
END as deployment_status
FROM farm_totals
) as Deployment
GROUP BY deployment_status;
"""


# Total hives in farm = total sensors in farm 
# Each hive grows by 0.2 frames per day
# Find how many days are between current date and commitment date (current date being latest hive_update for sensor)
# The above is the total days per sensor_id (hive) until commitment
# Multiply 0.2 by the above to get total frame increase per sensor
# on the groupby per farm, we sum the above
# add this sum to total frames per farm and re-evaluate deployment statuses

# Do not use hard coded values for date. I can just use 15-5 = 10
# Or do I need to find the latest hive update creation time for each hive, and find difference in dates for each?



result = run_query(query_5)
print(result)

  deployment_status  farm_count
0     over-deployed         157
1    under-deployed          27


## Part 2: Optimization Challenge (90 minutes)

### Question 6: Redeployment Plan

Suggest a new assignment plan that would:
- Minimize both over-deployment and under-deployment (prioritize fixing under-deployment)
- Move hives in groups of 4 only (called a "pallet") <span style="color: red;">  from the same orchard?? Assume yes </span>
- Involve as few farms and beekeepers as possible
- Require the smallest number of hive transfers

**Note**: You should use the projected deployment status from question 5.

### Initial Notes on Below:

Greedy algorithm
- makes the best choice (most greedy choice) at each step with the hope of finding a global optimum.
- eg. find farm with the biggest under-deployment (as this is priority) and then we will 
   fill this farm from eg the farm with the niggest over-deployment/ maybe take into account location if near farm
- trasnfer pallets in groups of 4 hives
- repeat until under deployment minimised or no more moves can be made

Linear:
- need to minimise a function, and define the conditions/constraints
- do i need to translare the problem to an optimisation algorithm and then i write it?
- Linear combination of all variables that we need to minimise:
- eg: (e.g., C1 * total_under_deployment + C2 * total_over_deployment + C3 * num_transfers + C4 * num_farms_involved + C5 * num_beekeepers_involved).
- number of beekeepers is from group_id
- sensor_id = unique hive
- constraints: Inequalities: linear inequalities (e.g., ax+by≤c) to define  constraints.

Variables:

Constraints:

Objective Function:

Heuristic:
- Greedy is an example of a heuristic approach, but i can do my own custom model

In [7]:
# Step 1: Implement optimization algorithm
# You can use:
# - Greedy algorithm
# - Linear programming (e.g., scipy.optimize) (#read linear programming from documentation)
# - Custom heuristic
     
# Your optimization code here

## Linear Programming using scipy.optimise.linprog

Objective Function:

$$
\min_x \; c^\top x
$$

## X
- Where the vector x is the decision variables. 
- Each element in x represents a pallet transfer from farm i to farm j.  
<br>

## C
- The vector c is the objective function coefficient.
- The coefficients will provide a weighting for each transfer:
    - Large negative coefficents make the optimiser want to choose the transfer
    - Large positive coefficents make the optimiser want to choose the transfer less
- Examples of weightings:
    - If the farm is under deployed, the coefficent will be smaller (more negative) as we prioritise under-deployed farms
    - If the farm is over deployed, the coefficent will be larger (less negative) as we do not prioritise over-deployed farms
    - Each coefficient will have a transfer cost, increasing the value of the coefficient, this will aid in minimising the values in x for total transfers.

The weightings for c will result in the optimiser selecting the combination of trasnfers with the lowest objective value (minimum transfers) while prioritising under-deployed farms.


## Bounds

All pallet trasnfers are equal or greater than 0.  
`bounds = [(0, None) for _ in range(n_vars)]` where n_vars is the total decision variables.




## Data needed:

- Over-Deployed farm: 
    - DF containing the farm id and its corresponding surplus in hives. 
    - Values will be positive as hives will be transfered FROM this farm
- Under-Deployed farm: 
    - DF containing the farm id and its corresponding surplus in hives. 
    - Values will be negative as hives will be transfered TO this farm
    

## Objective Coefficient Calculation:

In [8]:
## DO NOT RUN
c = np.zeros(n_vars)
var_idx = 0
for i, over_farm in over_deployed.iterrows():
    for j, under_farm in under_deployed.iterrows():
        # Weight heavily towards fixing under-deployment
        under_penalty = abs(under_farm['Surplus hives']) * 10  # Higher weight for under-deployment
        over_penalty = over_farm['Surplus hives'] * 1
        transfer_cost = 1  # Cost per pallet transfer

        c[var_idx] = -(under_penalty * 0.8) + (over_penalty * 0.2) + transfer_cost
        # 0.8 and 0.2 used here for a relative weighting, may not need it
        var_idx += 1

NameError: name 'np' is not defined

We can see that we have two separate weighting stages: (maybe make this class variable)
- under penalty * 10 and over_penalty * 1 (general weighting)
- under_penalty * 0.8 and over_penalty * 0.2 (relative weighting)  

These weightings allow for under-deployed farms to be prioritised (leeds to happier farmers). 
Transfer cost + 1 to increase each coefficient more to further minimise the total transfers in x.



## Constraints understanding:

$A_{supply}x <= b_{supply}$  



- Supply: Over deployed farms cannot give more pallets than is has surplus
- Equation: sum(trasnfers FROM farm i) <= max_available_pallets[i]
- Matric A_supply: 
    - Each row is an over-deployed farm. 
    - The total columns is the size of the decision variables array (array telling us how many trasnfers there are from a site i to j based off the equation var_idx = i * n_under + j)
    - The values of ther matrix A represent coefficients for the decision variables in the inequality. 1 representing FROM that farm (in that row) and 0 not.

A_supply = np.zeros((n_over, n_vars))


SAME FOR DEMAND

$A_{demand}x >= b_{demand}$ 

- Demand: Ensure each under -deployed farm receives at least minimum pallets. 
- min_pallets = int(abs(under_farm['surplus_deficit']) // pallet_size) (we want a minimum pallets to prioritise under-deployed farms)
- Equation: sum(transfers TO farm j) ≥ min_needed_pallets[j]
- linprog handles <= inequalities so need to multiply both sides by -1:
    - b_demand[j] = -min_pallets
    - each value in A is -1 instead of 1
- Matric A_demand: 
    - Each row is an under-deployed farm. 
    - The total columns is the size of the decision variables array (array telling us how many trasnfers there are from a site i to j based off the equation var_idx = i * n_under + j)
    - The values of the matrix A represent coefficients for the decision variables in the inequality. -1 representing a trasnfer TO that farm (in that row) and 0 not.

A_demand = np.zeros((n_under, n_vars))

## Matrix element calculation

In [ ]:
## DO NOT RUN
A_supply = np.zeros((n_over, n_vars))
b_supply = np.zeros(n_over)

for i, (idx, over_farm) in enumerate(over_deployed.iterrows()): # iterate over each over-deployed farm
    max_pallets = int(over_farm['Surplus hives'] // pallet_size) # round down using // operator so not too many pallets
    b_supply[i] = max_pallets # b is the vector containing the max pallets, Ax<=b

    print(f"\nFarm {over_farm['farm_id']} (row {i}) constraint:")
    print(f"  Can provide maximum {max_pallets} pallets")

    # Set coefficients for this farm's variables
    constraint_terms = []
    for j in range(n_under): 
    # iterates through each under-deployed farm for each over-deployed farm to allow transfer from over to under farm.
    # Note: there is no constraint on how many hives a farm gives another farm as long as its below the max_pallets per farm. (so all coeffcients are 1, there can be a trasnfer FROM a over to under farm)
        var_idx = i * n_under + j # row-major index formula explained below
        A_supply[i, var_idx] = 1 # assign 1 to where there will be a transfer in x[var_idx]
        to_farm = under_deployed.iloc[j]['farm_id'] # from farm i to farm j
        constraint_terms.append(f"x[{var_idx}](to Farm {to_farm})")

    print(f"  Constraint: {' + '.join(constraint_terms)} <= {max_pallets}")


## Proof of row-major index formula:


$$ \text{var_idx} = (i \times n_{\text{under}}) + j $$

Proof the formula never exceeds array bounds (decision variables x has size n_vars-1)  

- A needs to have total rows as e.g. total over-deployed farms and then the number of columns needs to match total variables.
- The formula ensures a mapping from A to x that allows A to have total columns matching the total variables, as the total variables vector x has variables for under and over deployed farms. (Ax<=b)

|      | $j=0$      | $j=1$      | ... | $j=n_{\text{under}}$ |
|:-----|:-----------|:-----------|:----|:---------------------|
| **$i=0$** | $X_{0,0}$  | $X_{0,1}$  | ... | $X_{0,n_{\text{under}}}$   |
| **$i=1$** | $X_{1,0}$  | $X_{1,1}$  | ... | $X_{1,n_{\text{under}}}$   |
| **...** | ...        | ...        | ... | ...                  |
| **$i=n_{\text{over}}$** | $X_{n_{\text{over}},0}$ | $X_{n_{\text{over}},1}$ | ... | $X_{n_{\text{over}},n_{\text{under}}}$ |


- Above is the Decision Variables matrix
- n_under = total farms under deployed
- n_over = total farms over deployed
- $x[i,j]$ = total transfer of pallets moved from farm i to farm j  
- To provide the decision variables vector x, we flatten the above matrix.
- The row-major index formula ensures the flattening occurs one row at a time, so first row of matrix and then second etc are appended to the the vector x

$x = [X_{0,0}, X_{0,1}, \dots, X_{0,n_{\text{under}}}, X_{1,0}, X_{1,1}, \dots, X_{1,n_{\text{under}}}, \dots, X_{n_{\text{over}},0}, X_{n_{\text{over}},1}, \dots, X_{n_{\text{over}},n_{\text{under}}}]$

### Proof
$x = [x_0, x_1, \dots, x_{n _\text{vars} - 1}]$  
               
Array size: $n_{\text{vars}} = n_{\text{over}} \times n_{\text{under}}$

Valid indices: $0$ to $(n_{\text{vars}} - 1)$

Maximum possible $\text{var_idx}$: occurs when $i = n_{\text{over}}-1$ and $j = n_{\text{under}}-1$

Maximum $\text{var_idx} = (n_{\text{over}}-1) \times n_{\text{under}} + (n_{\text{under}}-1)$
$$= n_{\text{over}} \times n_{\text{under}} - n_{\text{under}} + n_{\text{under}} - 1$$
$$= n_{\text{over}} \times n_{\text{under}} - 1$$
$$= n_{\text{vars}} - 1$$



                  
Each index in the decision variables array x refers to a trasnfer from farm i to j  

This is a row-major index used to store multidimensional arrays (2D) into linear storage (1D).  
Essentially flattening the matrix into a vector  

Needed a way to represent the matrices for constraints in the form of the decision variable so flattening was needed. It is a link from matrix A to x in Ax<=b.

# Surplus Query for Over and Under Deployed farms

In [9]:
surplus_hives_query = '''
WITH ranked_hive_updates as 
(
SELECT * from temp_days_to_comm
), farm_totals as 
( 
SELECT DISTINCT 
o.farm_id,
sds.orchard_id,
COUNT(sensor_id) as total_hives_in_farm,
SUM(bee_frames) as total_bee_frames_in_farm,
MAX(o.acres_pollinated * o.committed_frames_per_acre) as total_commitment
FROM ranked_hive_updates
JOIN sensor_daily_statuses as sds
	USING(sensor_id)
JOIN orchards as o
	ON sds.orchard_id = o.id
WHERE ranks = 1
AND sds.orchard_id is not NULL 
GROUP BY o.farm_id
)
SELECT 
farm_id,
orchard_id,
total_bee_frames_in_farm - total_commitment as "Surplus total frames (positive is over-deployed)",
total_commitment as total_commitment_frames,
total_hives_in_farm,
total_bee_frames_in_farm / total_hives_in_farm as average_hive_size_in_farm,
(total_bee_frames_in_farm - total_commitment) / (total_bee_frames_in_farm / total_hives_in_farm) as "Surplus hives",
CASE 
	WHEN total_bee_frames_in_farm > total_commitment THEN 'over-deployed'
	WHEN total_bee_frames_in_farm < total_commitment THEN 'under-deployed'
	WHEN total_bee_frames_in_farm = total_commitment THEN 'exact'
	ELSE NULL 
END as deployment_status
FROM farm_totals

'''

surplus_df = pd.read_sql_query(surplus_hives_query, conn)
surplus_df

,farm_id,orchard_id,Surplus total frames (positive is over-deployed),total_commitment_frames,total_hives_in_farm,average_hive_size_in_farm,Surplus hives,deployment_status
0,1651,4159.0,16850.399623,5280.0,1759,12.581239,1339.327506,over-deployed
1,1657,3907.0,4911.214788,1332.0,471,13.255233,370.511386,over-deployed
2,1667,4533.0,345.025431,532.0,75,11.693672,29.505310,over-deployed
3,1676,4078.0,12.056165,96.0,12,9.004680,1.338878,over-deployed
4,1678,4048.0,27063.973379,732.0,1907,14.575760,1856.779632,over-deployed
...,...,...,...,...,...,...,...,...
179,2437,5357.0,-2246.823085,3040.0,56,14.163873,-158.630553,under-deployed
180,2443,5369.0,-453.524636,504.0,6,8.412561,-53.910415,under-deployed
181,2446,5374.0,3696.659715,1508.0,563,9.244511,399.876175,over-deployed
182,2448,5376.0,-1277.786935,1800.0,37,14.113867,-90.534151,under-deployed


In [10]:
# Find over-deployed df: (Note there are no farms with 0 surplus)
over_deployed = surplus_df.loc[surplus_df['Surplus hives']>0, ['farm_id', 'Surplus hives', 'orchard_id', 'average_hive_size_in_farm']]
over_deployed

,farm_id,Surplus hives,orchard_id,average_hive_size_in_farm
0,1651,1339.327506,4159.0,12.581239
1,1657,370.511386,3907.0,13.255233
2,1667,29.505310,4533.0,11.693672
3,1676,1.338878,4078.0,9.004680
4,1678,1856.779632,4048.0,14.575760
...,...,...,...,...
176,2419,393.277676,5324.0,12.402195
177,2421,1040.451211,5321.0,12.197225
178,2435,220.221901,5354.0,11.021578
181,2446,399.876175,5374.0,9.244511


In [11]:
# Find under-deployed df:
under_deployed = surplus_df.loc[surplus_df['Surplus hives']<0, ['farm_id', 'Surplus hives', 'orchard_id', 'average_hive_size_in_farm']]
under_deployed

,farm_id,Surplus hives,orchard_id,average_hive_size_in_farm
13,1712,-3.040162,4424.0,17.738200
28,1751,-306.245684,4034.0,11.903413
30,1756,-83.424797,4637.0,10.414002
32,1767,-78.671838,4629.0,13.054332
33,1768,-44.591297,4588.0,12.580029
36,1778,-23.367121,4204.0,13.256086
45,1812,-133.137263,4208.0,14.389871
55,1863,-32.705585,3938.0,11.235582
56,1864,-6.543390,4760.0,7.653433
65,1892,-2.026213,4219.0,10.939623


In [21]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from datetime import datetime, timedelta

class RedeploymentOptimisation:
    
    def __init__(self, over_deployed, under_deployed):
        '''
        Initialise Optimisation
        '''
        self.over_deployed = over_deployed
        self.under_deployed = under_deployed
        self.pallet_size = 4
        self.under_deployed_weight = 10
    
    
    def setup_linear_program(self):
        """
        Set up the linear programming problem
            
        Returns:
            Tuple of (c, A_ub, b_ub, bounds)
        """
        over_deployed = self.over_deployed
        under_deployed = self.under_deployed
#         movable_hives = self.movable_hives # base on is_locked_to_group
        
        # Decision variables: x[i,j] = number of pallets (groups of 4 hives) to move from farm i to farm j
        n_over = len(over_deployed)
        n_under = len(under_deployed)
        n_vars = n_over * n_under
        
        # Objective function coefficients
        # Minimize: weighted sum of under-deployment penalty + over-deployment penalty + transfer costs
        c = np.zeros(n_vars)
        
        var_idx = 0
        for i, over_farm in over_deployed.iterrows():
            for j, under_farm in under_deployed.iterrows():
                # Weight heavily towards fixing under-deployment
                under_penalty = abs(under_farm['Surplus hives']) * self.under_deployed_weight  # Higher weight for under-deployment
                over_penalty = over_farm['Surplus hives'] * 1
                transfer_cost = 1  # Cost per pallet transfer
                
                c[var_idx] = -(under_penalty * 0.8) + (over_penalty * 0.2) + transfer_cost
                var_idx += 1
        
        # Constraints
        constraints = []
        
        # Supply constraints (cannot move more than available surplus)
        A_supply = np.zeros((n_over, n_vars))
        b_supply = np.zeros(n_over)
        
        for i, (idx, over_farm) in enumerate(over_deployed.iterrows()):
            # Maximum pallets that can be moved from this farm
            max_pallets = int(over_farm['Surplus hives'] // self.pallet_size)# floor round
            b_supply[i] = max_pallets
            
            # Set coefficients for this farm's variables
            for j in range(n_under):
                var_idx = i * n_under + j
                A_supply[i, var_idx] = 1
        
        # Demand constraints (must satisfy under-deployment needs)
        A_demand = np.zeros((n_under, n_vars))
        b_demand = np.zeros(n_under)
        
        for j, (idx, under_farm) in enumerate(under_deployed.iterrows()):
            # Minimum pallets needed for this farm
            min_pallets = int(abs(under_farm['Surplus hives']) // self.pallet_size) # floor round
            b_demand[j] = -min_pallets  # Negative because we're using <= constraints
            
            # Set coefficients for this farm's variables
            for i in range(n_over):
                var_idx = i * n_under + j
                A_demand[j, var_idx] = -1  # Negative because constraint is >= min_pallets
        
        # Combine constraints
        A_ub = np.vstack([A_supply, A_demand]) # vetrical stack for the constraint matrices
        b_ub = np.hstack([b_supply, b_demand]) # horizontal stack for the b vector
        
        # Bounds (non-negative integer variables)
        bounds = [(0, None) for _ in range(n_vars)]
        
        return c, A_ub, b_ub, bounds
    
    def solve_optimization(self):
        """
        Solve the optimization problem
        
        Returns:
            Dictionary containing optimization results
        """  
        data = [self.over_deployed, self.under_deployed]
        print(f"Found {len(self.over_deployed)} over-deployed farms")
        print(f"Found {len(self.under_deployed)} under-deployed farms")
#         print(f"Found {len(self.movable_hives)} movable hives")
        
        if len(self.over_deployed) == 0 or len(self.under_deployed) == 0:
            print("No optimization needed - no transfers possible")
            return {"status": "no_optimization_needed", "data": data}
        
        print("Setting up linear program...")
        c, A_ub, b_ub, bounds = self.setup_linear_program()
        
        print("Solving optimization...")
        try:
            # Use method='highs' for better integer handling (trasnfers are intergers and pallet size is an integer)
            result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
            
            if result.success:
                print("Optimization successful!")
                return self.interpret_results(result)
            else:
                print(f"Optimization failed: {result.message}")
                return {"status": "failed", "message": result.message, "data": data}
                
        except Exception as e:
            print(f"Optimization error: {str(e)}")
            return {"status": "error", "message": str(e), "data": data}
    
    def interpret_results(self, result):
        """
        Interpret optimization results
            
        """
        over_deployed = self.over_deployed
        under_deployed = self.under_deployed
        
        # Extract solution
        solution = result.x
        n_under = len(under_deployed)
        
        transfers = [] # note down details of each transfer to be printed
        total_pallets = 0
        estimate_total_frames = 0
        farms_involved = set()
        beekeepers_involved = set()
        
        var_idx = 0
        for i, (_, over_farm) in enumerate(over_deployed.iterrows()): 
            # we will iterate over every trasnfer, going through each row in the Decision variables matrix ($x[i,j]$)
            # so that we are iterating through each element in the decision variables vector x. 
            # (as we flattened the matrix to x as defined further above). 
            # We will find the total transfers of pallets from i to j.
            for j, (_, under_farm) in enumerate(under_deployed.iterrows()):
                pallets = round(solution[var_idx])
                estimate_frames = pallets * self.pallet_size * over_farm['average_hive_size_in_farm']
                if pallets > 0:
                    transfers.append({
                        'from_farm': over_farm['farm_id'],
                        'from_orchard': over_farm['orchard_id'],
                        'to_farm': under_farm['farm_id'],
                        'to_orchard': under_farm['orchard_id'],
                        'pallets': pallets,
                        'hives': pallets * self.pallet_size,
                        'frames': estimate_frames
                    })
                    total_pallets += pallets
                    estimate_total_frames += estimate_frames
                    farms_involved.add(over_farm['farm_id'])
                    farms_involved.add(under_farm['farm_id'])
                
                var_idx += 1
        
        return {
            "status": "success",
            "transfers": transfers, # details of each transfer made
            "summary": {
                "total_pallets_moved": total_pallets,
                "total_hives_moved": total_pallets * self.pallet_size,
                "farms_involved": len(farms_involved),
                "estimated_frames_transferred": estimate_total_frames,
                "objective_value": result.fun
            }, # total summary of all trasnfers
            "data": [self.over_deployed, self.over_deployed],
            "solution": solution
        }

    def print_results(self, results):
        """
        Print optimization results in a readable format
        
        Args:
            results: Results dictionary from solve_optimization
        """
        if results["status"] != "success":
            print(f"Optimization {results['status']}: {results.get('message', 'Unknown error')}")
            return
        
        print("\n" + "="*60)
        print("BEE FRAME REDEPLOYMENT OPTIMIZATION RESULTS")
        print("="*60)
        
        summary = results["summary"]
        print(f"Total pallets to move: {summary['total_pallets_moved']}")
        print(f"Total hives to move: {summary['total_hives_moved']}")
        print(f"Farms involved: {summary['farms_involved']}")
        print(f"Estimated frames transferred: {summary['estimated_frames_transferred']}")
        
        print("\nDETAILED TRANSFER PLAN:")
        print("-" * 60)
        
        for i, transfer in enumerate(results["transfers"], 1):
            print(f"Transfer {i}:")
            print(f"  From: Farm {transfer['from_farm']} (Orchard {transfer['from_orchard']})")
            print(f"  To: Farm {transfer['to_farm']} (Orchard {transfer['to_orchard']})")
            print(f"  Pallets: {transfer['pallets']} ({transfer['hives']} hives)")
            print(f"  Approximate frames: {transfer['frames']}")
            print()

In [22]:
optimizer = RedeploymentOptimisation(over_deployed, under_deployed)

# Solve optimization
results = optimizer.solve_optimization()

# Print results
# optimizer.print_results(results)

# # Access specific results
# if results["status"] == "success":
#     print(f"\nOptimization completed successfully!")
#     print(f"Recommended {results['summary']['total_pallets_moved']} pallet transfers")
#     print(f"Involving {results['summary']['farms_involved']} farms")

for x in results["solution"]:
    print(x)

Found 157 over-deployed farms
Found 27 under-deployed farms
Setting up linear program...
Solving optimization...
Optimization successful!
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
334.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
92.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
464.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
159.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
67.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
42.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
185.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
176.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
45.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
54.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2744.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
73.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.0
0.0
0.0
0.0
0.0
0.0
0.0

In [ ]:
# Step 2: Output your redeployment plan
# Format: DataFrame with columns:
# from_farm | to_farm | num_pallets | beekeeper | frames_moved

# Your output code here
    
transfers_df = pd.DataFrame(results['transfers'])
# can improve frames moved calculation
# need to do beekeepers
transfers_df

In [ ]:
# Step 3: Validate and summarize your solution
# - Verify all moves are in multiples of 4 hives
# - Check no farm goes negative
# - Calculate total improvement metrics

# Your validation code here
# - Verify all moves are in multiples of 4 hives
print("Checking all moves are multiples of 4:")
print(np.all(transfers_df['hives']%4 ==0))

# - Check no farm goes negative
# check total hives/frames in farm is not negative?
# Note that the total hive calculation is using an average of how many frames per hive (can be improved)
# Iterate through each transfer and create a post optimisation farm hive count

surplus_df
intial_farm_hives = surplus_df.loc[:,['farm_id', 'total_hives_in_farm']]
intial_farm_hives_dict = intial_farm_hives.to_dict()
intial_farm_hives_dict = intial_farm_hives.set_index('farm_id')['total_hives_in_farm'].to_dict()
post_optimisation_farm_hives_dict = intial_farm_hives_dict.copy()

for transfer in results['transfers']:
    from_farm = transfer['from_farm']
    to_farm = transfer['to_farm']
    hives = transfer['hives']
    post_optimisation_farm_hives_dict[from_farm] -= hives
    post_optimisation_farm_hives_dict[to_farm] += hives
    
print("Checking no farm now has a negative number of hives:")
print(np.all(np.array([*post_optimisation_farm_hives_dict.values()]) >=0))
    
# - Calculate total improvement metrics
# Compare initial and post optimisation frame counts in farms to the commitment.
# If the difference between post optimisation and commitment is smaller than the difference between initial frame counts 
# and commitment, then the optimisation has succesfully redeployed the pallets in the farms.

average_hive_size = surplus_df['average_hive_size_in_farm'].values
improvement_df = pd.DataFrame({'farm_id':surplus_df['farm_id'], 
                         'initial_hive_count':intial_farm_hives_dict.values(), 
                         'post_optimisation_hive_count':post_optimisation_farm_hives_dict.values(),
                         'initial_frame_count_average':np.array([*intial_farm_hives_dict.values()])*average_hive_size,
                         'post_optimisation_frame_count_average':np.array([*post_optimisation_farm_hives_dict.values()])*average_hive_size,
                         'commitment_to_frames':surplus_df['total_commitment_frames']})
improvement_df

initial_diff_to_commitment = sum(abs(improvement_df['initial_frame_count_average'] - improvement_df['commitment_to_frames']))
post_optimisation_diff_to_commitment = sum(abs(improvement_df['post_optimisation_frame_count_average'] - improvement_df['commitment_to_frames']))
print(f"The intial total differences between frame counts at all farms and the commitment frames was {initial_diff_to_commitment}")
print(f"The post optimisation total differences between frame counts at all farms and the commitment frames was {post_optimisation_diff_to_commitment}")

print(f"\n Total under delpoyed farms is {sum(improvement_df['post_optimisation_frame_count_average'] - improvement_df['commitment_to_frames']<0)}")
improvement_df

# Found 3 farms which have a large post optimisation frame count. Need to debug but no time :(
improvement_df[abs(improvement_df['post_optimisation_frame_count_average'] - improvement_df['commitment_to_frames'])>1000]

## INITIAL NOTES: Data needed for algorithm
- orchard data for all relevent farms, including how many frames needed and how many they have after projection
- get deployment status for each farm and merge with orchard requirements
- calculate surplus in frames (projected frames - commitment)
    - surplus > 0 is over-deployed
    - surplus <0 is under-deployed
- active sensors and their group_id (for beekeepers) and is_locked_to_group (maybe use this to limit beekeepers)
- if sensor is locked_to_group (sensors df) then im assuming the hive cannot be moved?? so find these hives if sensors active

**Decision variables: x[i,j] = number of pallets (groups of 4 hives) to move from farm i to farm j**  
n_over = len(over_deployed)  # i is over-deployed farms
n_under = len(under_deployed)   # j is under-deployed farms
n_vars = n_over * n_under

**Objective function coefficients:**
Minimize: weighted sum of under-deployment penalty + over-deployment penalty + transfer costs
c = np.zeros(n_vars)
- possibly give weights to each decision variable based on if surplus is less than 0, then higher weight as under-deployed

**Constraints:**
- Supply constraint: A_sx<=b_s where b_s is the max pallets to be moved from an oveer-deployed farm (total frames rounded down divided by 4) UNDERSTAND A
sum(transfers FROM farm i) ≤ max_available_pallets[i]
- Demand Constraint: A_dx <= b_d where b_d is the min pallets needed for the farm to get out of under-deployment (remember to change sifn so constraint is <=). UNDERSTAND A
sum(transfers TO farm j) ≥ min_needed_pallets[j]

**Bounds:**
 (non-negative integer variables)
bounds = [(0, None) for _ in range(n_vars)]

# After Thoughts

- Possible reason for large differences found above in the first iteration:
    - The farms with large under-deployment had too small (very negative) weightings/ coefficients in the objective function so too many were transfered to this farm
    - The farms with large over-deployment had too large weightings/ coefficients in the objective function so not enough were transferred from these farms.
    - Possible solution: reduce weightings
    - Check how many farms are now under-deployed compared to before
    - transfer cost doesnt really do much when values are so high (maybe normalise)
    - maybe create another inequality for maximum trasnfer to the farm, to limit over transfer
    - maybe a constraint for the final hives over or under the commitment being within a certain value, so we avoid large over deployed at the end.
    
- Definition in c/ the objective coefficients changes to :
    - If the farm is over deployed, the coefficent will be larger (less negative) as we do not prioritise over-deployed farm

- As i used linprog, the final decision variables may not be integers. I rounded the integers to attain integer transfers (maybe should have rounded down). However, when looking at results.x, the values seem to be integers (likley due to method = highs)
    - can use milp with the integrality argument to define the variables as integers
    
- Did not include:
    - Limiting total beekeepers
    - include the 'is_locked_to_group' data in the sensors table

- Used average hive sizes but possibly could have used exact
